In [1]:
## mmrotate1.x기반으로 진행 
# Check MMRotate installation
import mmrotate
print(mmrotate.__version__)
# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.0.0rc1
3.1.0
11.1
GCC 7.3


In [2]:
### rtm_det configuration load ###
from mmengine.config import Config
cfg = Config.fromfile('mmrotate/configs/rotated_rtmdet/rotated_rtmdet_l-coco_pretrain-3x-dota_ms.py')
### rtm_det configuration load ###

In [3]:
from mmengine.runner import set_random_seed

########### dataset loader setting ###################

# Modify dataset type and path
cfg.dataset_type = 'DOTADataset'
cfg.data_root = 'container_dataset/'

cfg.train_dataloader.dataset.type = cfg.dataset_type
cfg.train_dataloader.dataset.data_root = cfg.data_root
cfg.train_dataloader.dataset.data_prefix.img_path = 'train_images'
cfg.train_dataloader.dataset.ann_file = 'train_lbl'
### doota dataset에서 class 1개로 변경 
cfg.train_dataloader.dataset.metainfo = {
        'classes':
        ('container',),
        # palette is a list of color tuples, which is used for visualization.
        'palette': [(165, 42, 42),]
    } 
                            

cfg.val_dataloader.dataset.type = cfg.dataset_type
cfg.val_dataloader.dataset.data_root = cfg.data_root
cfg.val_dataloader.dataset.data_prefix.img_path = 'train_VAL_patch_img'
cfg.val_dataloader.dataset.ann_file = 'train_VAL_patch_lbl'
### doota dataset에서 class 1개로 변경 
cfg.val_dataloader.dataset.metainfo = {
        'classes':
        ('container',),
        # palette is a list of color tuples, which is used for visualization.
        'palette': [(165, 42, 42),]
    } 
                            

cfg.test_dataloader.dataset.type = cfg.dataset_type
cfg.test_dataloader.dataset.data_root = cfg.data_root
### doota dataset에서 class 1개로 변경 
cfg.test_dataloader.dataset.metainfo = {
        'classes':
        ('container',),
        # palette is a list of color tuples, which is used for visualization.
        'palette': [(165, 42, 42),]
    } 
########### dataset loader setting ###################


##### 모델 head 수정 및 pretrained 안사용하도록 처리 ####
# 예측할 class가 1개 이기때문에 수정
cfg.model.bbox_head.num_classes = 1

#backbone pretrain 모델 사용 '안' 하도록 set
cfg.model.backbone.init_cfg = None
cfg.model.neck.init_cfg = None
cfg.model.init_cfg = None
cfg.model.bbox_head.init_cfg = None
cfg.coco_ckpt=''
cfg.checkpoint=''
##### 모델 head 수정 및 pretrained 안사용하도록 처리 ####

In [4]:
# Set up working dir to save files and logs.
cfg.work_dir = './VER1'

In [5]:
### log가 iteration 마다 얼마나 표시할지 ( 처음에 학습경향볼때는 1로 두고 그이후에는 1epoch 마다 찍도록 set)
cfg.default_hooks.logger.interval = 100

### checkpoint 10 개단위로 150개 저장하도록 진행 (총 300개 생기는데 그중에 100개)
cfg.default_hooks.checkpoint.max_keep_ckpts = 300
cfg.default_hooks.checkpoint.interval = 10
#max epoch
cfg.max_epochs = 3000
cfg.train_cfg.max_epochs = 3000

cfg.optim_wrapper.optimizer = dict(type='AdamW', lr=cfg.base_lr, weight_decay=0.05)
cfg.param_scheduler[1].eta_min = cfg.param_scheduler[1].eta_min/30 ## 대략 1000epch 단위로 0.1 씩 떨어지게 진행

#cosine scheduler 시작은 90epoch 부터 진행하고 마지막 epoch 까지 천천히 계속 learning rate 감소시킴 
cfg.param_scheduler[1].begin = 90
cfg.param_scheduler[1].end = 3000
cfg.param_scheduler[1].T_max = 2910

### epoch 단위이기때문에 10epohc마다 찍도록 
cfg.interval = 10
cfg.train_cfg.val_interval=10

###  batchsize
cfg.train_dataloader.batch_size=2

### seed 고정 
cfg.seed = 199002
set_random_seed(0, deterministic=False)

cfg.device='cuda'

########################### train Data augmentation #################################
cfg.train_pipeline = [
    dict(type='mmdet.LoadImageFromFile', backend_args=None),
    dict(type='mmdet.LoadAnnotations', with_bbox=True, box_type='qbox'),
    dict(type='ConvertBoxType', box_type_mapping=dict(gt_bboxes='rbox')),
    dict(
        type='mmdet.RandomResize',
        ### multisclae로 학습진행
        resize_type='mmdet.Resize',
        scale=(
            1024,
            1024,
        ),
        ratio_range=(
            3.0,
            5.0,
        ),
        keep_ratio=True),
    dict(
        type='RandomRotate',
        prob=0.5,
        angle_range=180,
        rect_obj_labels=[
            9,
            11,
        ]),
    dict(type='mmdet.RandomCrop', crop_size=(
        1024,
        1024,
    )),
    dict(type='mmdet.YOLOXHSVRandomAug'),
    dict(
        type='mmdet.RandomFlip',
        prob=0.75,
        direction=[
            'horizontal',
            'vertical',
            'diagonal',
        ]),

    dict(
        type='mmdet.Pad',
        size=(
            1024,
            1024,
        ),
        pad_val=dict(img=(
            114,
            114,
            114,
        ))),
    dict(type='mmdet.PackDetInputs'),
]
########################### train Data augmentation #################################

#valset은 현재 5120,5120으로 resize 이후 1024,1024로 patch 단위로 자른 이미지를 사용하고 있기 때문에 resize없이 test진행
#실제 test시에는 1024 기준 3배 ,4배 ,5배 (.5배도?) 로 키운이후에 1024 patch 단위로 예측하는 형태로 test 진행해볼 예정 
cfg.val_pipeline = [
    dict(type='mmdet.LoadImageFromFile', backend_args=None),
    dict(type='mmdet.Resize', scale=(
        1024,
        1024,
    ), keep_ratio=True),
    dict(type='mmdet.LoadAnnotations', with_bbox=True, box_type='qbox'),
    dict(type='ConvertBoxType', box_type_mapping=dict(gt_bboxes='rbox')),
    dict(
        type='mmdet.Pad',
        size=(
            1024,
            1024,
        ),
        pad_val=dict(img=(
            114,
            114,
            114,
        ))),
    dict(
        type='mmdet.PackDetInputs',
        meta_keys=(
            'img_id',
            'img_path',
            'ori_shape',
            'img_shape',
            'scale_factor',
        )),
]

# 일단 validation size로 정의해뒀으나 실제로 test 진행시에는 multisize로 진행예정
cfg.test_pipeline = [
    dict(type='mmdet.LoadImageFromFile', backend_args=None),
    dict(type='mmdet.Resize', scale=(
        1024,
        1024,
    ), keep_ratio=True),
    dict(
        type='mmdet.Pad',
        size=(
            1024,
            1024,
        ),
        pad_val=dict(img=(
            114,
            114,
            114,
        ))),
    dict(
        type='mmdet.PackDetInputs',
        meta_keys=(
            'img_id',
            'img_path',
            'ori_shape',
            'img_shape',
            'scale_factor',
        )),
]
########################### train Data augmentation #################################
cfg.train_dataloader.dataset.pipeline = cfg.train_pipeline
########################### train Data augmentation #################################

cfg.val_dataloader.dataset.pipeline = cfg.val_pipeline
cfg.test_dataloader.dataset.pipeline = cfg.test_pipeline

cfg.evaluation = dict(save_best="mAP")

# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

Config:
default_scope = 'mmrotate'
default_hooks = dict(
    timer=dict(type='IterTimerHook'),
    logger=dict(type='LoggerHook', interval=100),
    param_scheduler=dict(type='ParamSchedulerHook'),
    checkpoint=dict(type='CheckpointHook', interval=10, max_keep_ckpts=100),
    sampler_seed=dict(type='DistSamplerSeedHook'),
    visualization=dict(type='mmdet.DetVisualizationHook'))
env_cfg = dict(
    cudnn_benchmark=False,
    mp_cfg=dict(mp_start_method='fork', opencv_num_threads=0),
    dist_cfg=dict(backend='nccl'))
vis_backends = [
    dict(type='LocalVisBackend'),
]
visualizer = dict(
    type='RotLocalVisualizer',
    vis_backends=[
        dict(type='LocalVisBackend'),
    ],
    name='visualizer')
log_processor = dict(type='LogProcessor', window_size=50, by_epoch=True)
log_level = 'INFO'
load_from = None
resume = False
custom_hooks = [
    dict(type='mmdet.NumClassCheckHook'),
    dict(
        type='EMAHook',
        ema_type='mmdet.ExpMomentumEMA',
        momentum=0.0002,
 

In [6]:
cfg.dump(file='custom_CONFIG_HJS_VER1.py')

In [7]:
## mmrotate 1.x 버전에서는 seed 고정하려면 mmdetection 3.x와 같이  randomness.seed=2023 randomness.diff_rank_seed=True randomness.deterministic=True configuration options으로 추가해야함
!python mmrotate/tools/train.py  custom_CONFIG_HJS_VER1.py --work-dir ver1 --resume --cfg-options randomness.seed=2023 randomness.diff_rank_seed=True randomness.deterministic=True

07/22 03:05:36 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.8.17 (default, Jul  4 2023, 06:27:59) [GCC 12.2.0]
    CUDA available: True
    numpy_random_seed: 2023
    GPU 0: NVIDIA GeForce RTX 3060
    CUDA_HOME: None
    GCC: gcc (Debian 12.2.0-14) 12.2.0
    PyTorch: 1.8.0+cu111
    PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v1.7.0 (Git Hash 7aed236906b1f7a05c0917e5257a1af05e9ff683)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.1
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=comp